In [1]:
from tensorflow.python.keras.layers import Bidirectional, Dropout, Activation, Dense, LSTM,GRU
 
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.callbacks import EarlyStopping
 
from sklearn.preprocessing import MinMaxScaler
 
from datetime import datetime

from binance.client import Client

from coinmarketcapapi import CoinMarketCapAPI, CoinMarketCapAPIError

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json

In [2]:
client = Client('', '')

In [3]:
cmc = CoinMarketCapAPI('  ')

In [4]:
info = client.get_all_tickers()

In [5]:
ticker_list = [d['symbol'] for d in info if d['symbol'].endswith('USDT')]

In [6]:
rank_info = cmc.cryptocurrency_map(sort='cmc_rank').data

In [7]:
target_rank_info = [d['symbol']+'USDT' for d in rank_info if d['rank'] >= 300 and d['rank'] <= 500]

In [8]:
target_ticker_list = [d for d in ticker_list if d in target_rank_info]

In [12]:
def is_candidate(filtered_klines_info, tolerable_volume):
    context_klines_info = filtered_klines_info[:len(filtered_klines_info) -1]
    is_candidate = False
    trigger_kline = filtered_klines_info[-1]
    
    for context_kline in context_klines_info:
        
        volume_gate = float(context_kline[3])  <  tolerable_volume * 1.2
        if volume_gate :
            is_candidate = True
        else:
            return False

    print('Trigger Line')
    print(trigger_kline[-1])
    

    trigger_gate = float(trigger_kline[3]  > tolerable_volume * 10) 


    if trigger_gate:
        is_candidate = True
    else:
        is_candidate = False

    return is_candidate

In [13]:
def find_percentage_increase_or_decrease(focused_kline_array):
    percentage_value = []
    for kline in focused_kline_array:
        diff = kline[2] - kline[1]
        perc_diff = diff/kline[1]
        percentage_value.append(perc_diff*100)
    return percentage_value 

In [14]:
hit_ticker_dictionary = {}

In [15]:
full_test_data_set = []

for target_ticker in target_ticker_list:

    klines = client.get_historical_klines(target_ticker, Client.KLINE_INTERVAL_15MINUTE, "7 day ago UTC") 
    length = int(len(klines))
    if(length < 24):
        break
    
    array = np.array(klines, dtype=np.float64)
    focused_kline_array = np.delete(array, [2, 3, 6, 7, 8, 9, 10, 11], 1)
    percentage_value = find_percentage_increase_or_decrease(focused_kline_array)
    percentage_value = np.reshape(percentage_value, (length, 1))
    focused_kline_array = np.append(focused_kline_array, percentage_value, 1)
    average_volume = np.mean(focused_kline_array[:,3])
    tolerable_volume = average_volume * 1.2
    
   
    for x in range(0, length):
        
 
        filtered_klines_info = focused_kline_array[x:x+24]
        buy_kline_info = filtered_klines_info[-1]
        if(len(filtered_klines_info) == 24 and is_candidate(filtered_klines_info, tolerable_volume)):
            
            
            print('********** SUCCESS ************************')
            print(target_ticker)
            print(x)
            print(buy_kline_info)
            print(klines[x:x+24][-1])
            
            key_test_data_set = [] 
            key_test_data_set.append(buy_kline_info[0])
            key_test_data_set.append
            
            
            hit_ticker_dictionary[target_ticker] = filtered_klines_info[-1:]
            

0.7812500000000043
-0.2590673575129608
0.4087193460490391
0.6802721088435381
1.6172506738544374
0.26525198938992806
-0.2649006622516448
0.930851063829777
-0.2635046113306876
1.0568031704095049
-0.6535947712418307
0.0
-0.2635046113306876
-1.1873350923483006
0.0
0.4010695187165705
0.2663115845539357
-0.26595744680851824
-0.6666666666666673
0.40268456375840084
0.5347593582887669
-0.5312084993360046
-0.5347593582887854
-0.13458950201884637
0.26954177897573023
0.13495276653171778
-0.4043126684636234
1.4864864864864915
-0.9308510638297953
-0.5361930294906134
0.1345895020188277
-1.209677419354836
-0.1358695652173952
0.9549795361527861
0.0
0.13123359580052868
0.1308900523560247
0.3921568627450912
0.26075619295957214
-0.13003901170351478
-0.2604166666666561
0.0
1.820546163849153
0.0
-0.3759398496240535
-0.3773584905660311
0.505050505050502
0.3768844221105461
-0.12531328320802365
0.12547051442911278
0.12531328320802365
0.25031289111389954
-0.12500000000000358
0.25031289111389954
0.37406483790524

0.0
0.8658008658008665
0.3846153846153849
0.1915708812260538
0.191204588910134
0.0
0.0
0.957854406130269
-0.1897533206831121
-1.901140684410648
1.9379844961240327
0.1901140684410648
-0.18939393939393956
0.0
0.18939393939393956
0.18867924528301902
0.0
-0.18832391713747662
-0.18867924528301902
0.0
0.0
0.0
0.37807183364839353
0.0
-0.18867924528301902
-0.18939393939393956
0.7518796992481209
0.37243947858473025
0.0
0.18587360594795554
0.7421150278293142
0.18832391713747662
-0.18796992481203023
0.0
0.19455252918287955
-0.7751937984496131
0.0
0.19493177387914246
0.0
-0.19379844961240328
-0.3883495145631071
0.19455252918287955
0.0
0.0
-0.19379844961240328
-0.19379844961240328
0.19417475728155356
0.0
0.13598041881968442
0.2179242713157231
-0.10872519706442232
0.10890280424721202
-0.05446623093681459
-0.16353229762877977
0.38199181446112207
-0.35326086956521713
0.054659743099214696
-0.054629882545759775
0.0820344544708768
0.0
-0.2185792349726829
0.0
0.32885722115648924
0.0
0.2731494127287515
0.2

0.0
-0.14727540500736389
-0.1474926253687317
0.4437869822485211
0.36845983787767983
0.29411764705882376
0.44020542920029393
0.8645533141210382
0.142857142857143
0.49928673323823947
-0.1403508771929826
2.2456140350877214
-0.0669344042837945
-0.33444816053512477
0.5369127516778528
-0.13360053440213773
-0.33467202141900226
-0.06715916722633392
0.47043010752688963
-0.5344021376085509
-0.33579583613163966
-0.6738544474393537
-0.4736129905277331
0.9517335146159083
0.4901960784313808
0.4184100418410046
-0.41724617524339397
0.7681564245810141
-0.41580041580041616
-0.2785515320334264
-0.06988120195668147
-1.11966410076977
0.56617126680821
0.45766590389015743
0.45454545454546763
0.3389830508474674
0.5630630630630635
1.1185682326621935
0.0
0.3318584070796555
-0.22026431718062306
0.11037527593819302
-0.22075055187638604
-0.22099447513811254
0.4419889502762404
-0.10964912280702069
-0.10976948408342796
0.10989010989011304
-0.21953896816685592
-0.659340659340663
0.2212389380531037
-0.1103752759381930

0.5865102639296288
1.1684518013631968
0.38461538461538564
-0.2906976744185928
-0.29097963142580513
0.09737098344692885
0.0972762645914526
0.1947419668938746
0.0
0.0972762645914526
0.0971817298347871
-0.29097963142580513
-0.09737098344692885
0.0
-0.09746588693956718
0.09775171065494945
0.48971596474044754
0.38986354775828563
-0.2906976744185928
-0.19398642095052557
0.0972762645914526
-0.19493177387913435
-0.19512195121952114
0.3910068426197469
-0.09756097560975212
0.19550342130988194
0.7812499999999851
-0.1936108422071725
0.0
-0.3875968992248072
0.19455252918288835
0.3875968992248072
0.0
0.1932367149758543
0.09643201542911854
0.48123195380172956
0.0
0.0
-0.38461538461538564
1.1571841851494558
0.0
0.19212295869355606
-0.28763183125599723
0.2683363148479318
-0.08920606601248522
-0.3568242640499564
-0.2685765443151189
-0.0895255147717063
-0.5376344086021442
-0.08620689655172063
-0.3445305770887175
0.2590673575129578
0.3442340791738391
0.257731958762891
0.0
-0.2577319587628761
-0.0861326442

0.6877916973716421
-0.048709206039939563
-0.3164556962025272
0.24485798237021528
-0.2931118710307648
0.0
0.19607843137254105
-0.17106549364614032
0.17169487368163017
0.0
0.17156862745096488
0.07333170373990541
-0.09789525208027013
0.0
0.09775171065494945
0.0
0.0
0.12171372930866951
-0.12162490878132189
0.14641288433381539
0.5863669679941294
-0.2421307506053338
0.3635482307319542
0.38806694154741794
0.4122211445198921
0.09675858732462111
0.48390999274134716
-0.43279634527530564
-0.6776379477250786
0.17031630170316453
0.2905569007263804
-0.6032818532818538
-0.21849963583393978
-0.2430724355858115
0.3897685749086321
-0.5577109602327862
0.09763241396143121
-0.4869734599464468
-0.02450980392157613
0.024515812699198485
0.0
0.12215978499878188
0.39052965584574184
0.09746588693956718
0.24378352023402225
0.3649635036496455
0.1943634596695742
-0.24201355275896141
1.285471743875808
0.526567735758748
0.2856462746965057
0.14275517487509992
0.0
0.09521542489882973
0.0
-0.8573469873779451
-0.16786570

...

-0.30686612965094
0.35785288270377325
-0.3165809259992088
0.3979307600477609
0.11881188118812332
-0.6361829025844936
-0.080032012805131
0.23999999999999136
0.039872408293456535
-0.15974440894568703
0.7205764611689268
0.31834460803820164
-0.11914217633040786
-0.47713717693837016
0.3598560575769651
1.1947431302269933
-0.27548209366391646
0.11857707509881872
-0.35516969218626276
0.6336633663366342
-0.43273013375293773
-0.8681925808997553
-0.03980891719744785
0.8376545672118032
-0.5529225908372921
0.15885623510722813
-0.4361617763679491
-0.5571030640668441
0.640256102440977
-0.3972983710766701
-0.23904382470118662
-0.1980982567353365
0.039682539682535316
-0.27766759222531207
-0.19872813990460628
0.6767515923566841
0.2369668246445412
-0.35447026388343206
0.2369668246445412
-0.15772870662460584
-0.11848341232227938
0.3558718861209924
-0.23640661938533428
-0.15816528272044303
0.0
-0.35629453681709805
0.3178386968613432
0.35643564356435237
0.3550295857988125
-0.07858546168957876
-0.46929996089

0.5899705014749227
-0.5865102639296152
0.0
0.5899705014749227
-0.2923976608187218
-0.8771929824561452
0.5882352941176435
-0.8771929824561452
-0.5882352941176638
0.591715976331378
-0.29498525073747156
-0.29325513196479747
0.0
0.0
0.0
0.0
-0.5882352941176638
-0.29498525073747156
0.2941176470588115
0.0
0.2932551319648178
-0.2923976608187218
0.2932551319648178
0.2923976608187015
0.583090379008763
0.0
0.2915451895043815
1.1627906976744116
-0.28818443804035404
0.2898550724637563
-0.5763688760806881
0.0
-0.8695652173913091
0.0
-0.2923976608187218
0.0
0.0
0.0
0.0
0.2932551319648178
0.2923976608187015
0.0
0.0
0.0
0.0
0.2923976608187015
0.2923976608187015
0.0
0.5813953488372058
0.0
-0.5780346820809213
0.290697674418613
0.8746355685131244
-0.5797101449275528
0.5813953488372058
-0.5797101449275528
-0.2915451895043613
0.0
0.0
-1.1695906432748668
-0.5934718100890172
-0.5970149253731307
0.0
-0.3003003003003089
0.0
-0.9009009009009058
0.6079027355623063
-0.6042296072507516
0.0
0.0
-0.30211480362536536

0.17574692442882267
0.0
0.34965034965035
0.5226480836236939
1.0416666666666676
-0.17182130584192457
-0.5163511187607578
-0.17301038062283752
0.34662045060658614
-0.6908462867012096
0.3484320557491293
-0.17361111111111127
-0.17391304347826103
-0.8695652173913052
0.3502626970227674
0.8726003490401405
0.0
0.3448275862068969
0.0
-0.5163511187607578
0.5181347150259072
0.3442340791738385
0.0
-0.17182130584192457
-0.17241379310344845
0.6896551724137938
0.0
0.32948929159802337
0.0
-0.16393442622950835
0.16447368421052647
0.0
-0.16447368421052647
-0.16474464579901169
0.4942339373970351
0.4918032786885251
0.16181229773462796
-0.3231017770597741
-0.3236245954692559
0.32414910858995166
0.16181229773462796
-0.16181229773462796
-0.3236245954692559
-0.32467532467532495
0.0
-0.980392156862746
-0.16977928692699507
0.17006802721088451
0.5093378607809852
0.337268128161889
0.0
0.0
0.0
-0.5042016806722693
-0.5067567567567572
-1.020408163265307
-0.15243902439024404
-0.07621951219513047
-0.22900763358779494


0.7960808328230189
-0.18214936247723823
-0.7299270072992707
-0.12254901960784324
0.490196078431373
-0.24449877750611268
0.36764705882352977
-0.30506406345333226
0.061236987140225956
0.18371096142069146
1.1613691931540422
-0.42270531400965555
-0.606796116504855
-0.24420024420024442
0.0
0.12217470983506426
-0.42682926829267653
0.6740196078431446
0.12172854534388325
0.0
0.9118541033434591
0.6020469596628543
-0.4787552363853984
-0.4816375677302834
0.24169184290030232
-0.4216867469879455
0.0
0.3627569528415965
-0.6630500301386449
-0.12128562765312322
0.0
-0.6674757281553337
-0.061012812690671867
0.1833740831295913
-0.3052503052502987
-0.4892966360856273
-0.12277470841006763
0.49200492004920093
-0.30599755201957735
0.3692307692307696
-0.18393623543838833
0.4302397049784816
0.1840490797546082
-0.12262415695892101
0.0
0.0
0.12285012285012298
0.061236987140225956
-0.12254901960784324
-0.1835985312117437
-0.3067484662576622
0.3687768899815615
-0.1226993865030676
-0.12285012285012298
0.0615384615

0.39403803323624614
0.19053986294499728
0.11673281526198745
-0.05996402158705227
-0.12333333333332716
0.25032542304997446
0.42642502581870384
-0.1392988623926158
-1.0294899043570642
-0.2850149213694187
0.339632793059389
-0.07702354241318375
-0.4993632280983952
0.0572544793210304
0.01008945987759131
0.2555051269120803
-0.36216089333019347
0.07065948855988828
0.339324710230121
-0.2982473777688422
0.29263370332996363
-0.06036217303823389
-0.3087248322147572
0.49514955537591854
0.10045876167831998
0.05019912318864277
0.03345600535295717
0.6389455725420744
0.13312920189043848
0.08305923784844281
0.13285946789783487
0.023219557501562006
-0.03315430011272097
0.039787798408491046
0.43086305183614726
0.023116805917888766
1.221283337734348
-0.20365929770389055
0.006583928630220233
0.27332301511509116
0.1378766988378885
0.04915293115312235
-0.39305601048150485
0.3812027604337806
-0.271837028788523
0.1577546258257587
-0.18371497933206646
0.11167679421908588
-0.6595353720960613
0.38015272222405355


In [ ]:
full_test_data_set = []

for key in hit_ticker_dictionary.keys():
    key_test_data_set = []
    key_test_data_set.append(key) 
    trigger_time_in_ms = hit_ticker_dictionary[key][0][0]
    key_test_data_set.append(trigger_time_in_ms) 
    open_price = hit_ticker_dictionary[key][0][4]
    key_test_data_set.append(open_price)
    analysis_kline_list = client.get_historical_klines(symbol=key, 
                                                       interval=Client.KLINE_INTERVAL_1HOUR, 
                                                       start_str=trigger_time_in_ms)[1:6]
    for sell_kline in analysis_kline_list:
        close_price = sell_kline[4]

        key_test_data_set.append(close_price)
        diff = (float(close_price) - float(open_price))
        profit = diff/float(open_price) * 100
        key_test_data_set.append(profit)
        
    full_test_data_set.append(key_test_data_set)
    

In [ ]:
full_test_data_set